In [1]:
import osmium
import csv
from geopy.distance import geodesic


In [2]:

MIN_DISTANCE = 20 # meters for sampling way nodes


In [3]:

class NamedOnlyHandler(osmium.SimpleHandler):
    def __init__(self, outfile):
        super().__init__()
        self.writer = csv.writer(open(outfile, "w", newline=""))
        self.writer.writerow(["lat", "lon", "name"])  # header

    def node(self, n):
        name = n.tags.get("name")
        if name:
            self.writer.writerow([int(n.location.lat*1e6), int(n.location.lon*1e6), name])

    def way(self, w):
        name = w.tags.get("name")
        if name:
            last_point = None
            for n in w.nodes:
                lat, lon = n.lat, n.lon
                if last_point is None or geodesic((last_point[0], last_point[1]), (lat, lon)).meters >= MIN_DISTANCE:
                    self.writer.writerow([int(lat*1e6), int(lon*1e6), name])
                    last_point = (lat, lon)

In [4]:

handler = NamedOnlyHandler("saved_csv/p7.csv")


In [5]:
handler.apply_file("kanpur.osm.pbf", locations=True, idx="sparse_mem_array")
